# Preprocessing

In [362]:
import numpy as np
import pandas as pd
import os
import pickle
import utm

In [363]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc

In [364]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mineralogy

**To do**
* ~~Clean last points in Excel file while using "sum" as check~~

In [365]:
mineralogy = pd.read_excel("../_CLEANED/Vistelius_data_cleaned.xlsx", index_col=0)

### Check for wrong entries

In [366]:
np.isclose(mineralogy.loc[:, :"oth"].sum(axis=1), mineralogy.loc[:, "sum"])

array([ True,  True,  True, ...,  True,  True,  True])

In [367]:
wrong_sum = mineralogy.loc[~np.isclose(mineralogy.loc[:, :"oth"].sum(axis=1), mineralogy.loc[:, "sum"]), :]

In [368]:
# Check to see if any remaining incorrect lines are present
wrong_sum

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,l.i.,oth,sum,hs


In [369]:
mineralogy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4659 entries, 1 to 4659
Data columns (total 15 columns):
SiO2     4659 non-null float64
TiO2     4626 non-null float64
Al2O3    4659 non-null float64
Fe2O3    4657 non-null float64
FeO      4659 non-null float64
MnO      4544 non-null float64
MgO      4658 non-null float64
CaO      4659 non-null float64
Na2O     4659 non-null float64
K2O      4659 non-null float64
P2O5     3834 non-null float64
l.i.     4659 non-null float64
oth      874 non-null float64
sum      4659 non-null float64
hs       2240 non-null float64
dtypes: float64(15)
memory usage: 582.4 KB


In [370]:
wrong_sum.loc[:, :"oth"].sum(axis=1)

Series([], dtype: float64)

In [371]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,l.i.,oth,sum,hs
1,80.80,0.04,10.16,0.61,1.72,NaN,0.40,0.55,2.00,3.59,NaN,0.35,NaN,100.22,NaN
2,80.00,0.10,10.10,0.17,0.56,0.02,0.40,0.35,2.30,5.10,0.05,0.50,NaN,99.65,NaN
3,79.92,0.05,9.89,0.16,1.73,0.02,0.12,0.14,0.75,6.15,NaN,1.02,0.08,100.03,0.30
4,79.65,0.04,9.64,1.15,0.75,0.10,0.45,0.67,3.71,4.25,NaN,0.26,NaN,100.67,0.01
5,79.18,0.08,10.24,0.64,2.60,0.04,0.05,1.25,1.52,3.08,0.01,1.75,0.16,100.60,0.28


### Cleaning
**To do**
* ~~Replace zero values~~


In [372]:
# Would not do this this way since it becomes less clear what the variable means
# You should also replace 'minralogy' in all remaining cells by 'x' if you would want to do this
# x = mineralogy

In [373]:
# Replace zero values
mineralogy = preproc.replace_zero(mineralogy, 0.01)

In [374]:
mineralogy.to_excel("../_CLEANED/Vistelius_data_cleaned.xlsx")

* ~~Replace nan values~~

In [375]:
# Also replace NaN values by 0.01
mineralogy = preproc.replace_nan(mineralogy, 0.01)

* ~~Normalize~~

In [376]:
# Normalize specific columns
mineralogy.loc[:, :"oth"] = preproc.normalize(mineralogy.loc[:, :"oth"])# , total=mineralogy['sum'])

In [377]:
# Renew 'sum' column to reflect changes applied during cleaning
mineralogy["sum"] = mineralogy.loc[:, :"oth"].sum(axis=1)

In [378]:
# Check that sum of all variables + 'sum' == 200
assert all(np.isclose(mineralogy.loc[:, :'sum'].sum(axis=1), 200.0))

In [379]:
mineralogy.to_excel("../_INTERPOLATION/normalised_values.xlsx")

#### implementing normalised values in full_with_Coordinates

### centred log-ratio (clr) transformation

In [380]:
mineralogy_clr = preproc.clr(mineralogy)
mineralogy_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,l.i.,oth,sum,hs
1,5.116216,-2.494637,3.042697,0.229942,1.266563,-3.880932,-0.192052,0.126402,1.417386,2.002391,-3.880932,-0.325583,-3.880932,5.331906,-3.878435
2,5.027136,-1.657475,2.957645,-1.126847,0.065291,-3.266913,-0.271181,-0.404712,1.478019,2.274350,-2.350623,-0.048037,-3.960060,5.246874,-3.963466
3,4.900117,-2.476641,2.810616,-1.313490,1.067213,-3.392932,-1.601172,-1.447021,0.231409,2.335543,-4.086079,0.538894,-2.006637,5.124661,-0.684482
4,4.911676,-2.684842,2.799955,0.673796,0.246352,-1.768551,-0.264473,0.133557,1.845066,1.980953,-4.071136,-0.813039,-4.071136,5.146081,-4.064260
5,4.525075,-2.372377,2.479653,-0.292936,1.108863,-3.065524,-2.842381,0.376495,0.572062,1.278281,-4.451819,0.712967,-1.679230,4.764504,-1.113632


### Principal Component Analysis (PCA)

##### code doesn't run with NaN values --> very small value is given to the NaN value: (0.000001)
**OK, let's do this step after replacing the zeroes and also use 0.01 as replacement to not introduce high variance effects**  
**I've written a small function for this, similar in syntax to the 'replace_zero' one (see code above)**  
**If this would seem to be too much of a 'correction' of the data, you could leave out certain rows too get rid of NaN values**  
**You could also drop certain columns which have a very high number of NaN values, such as 'oth' e.g.**

In [381]:
mineralogy_pca = preproc.pca(mineralogy_clr)
preproc.pca_variance(mineralogy_pca)

9 PCA components  out of 15 components with variance sum 0.966625886419962 needed for obtaining sum of variance > 0.95


array([2.72769107e-01, 1.98486337e-01, 1.35269270e-01, 1.11034812e-01,
       8.48676592e-02, 5.96963124e-02, 4.62857519e-02, 3.20709944e-02,
       2.61456423e-02, 1.88742451e-02, 1.07280680e-02, 3.03849513e-03,
       7.04237437e-04, 2.90679570e-05, 1.71494552e-32])

In [382]:
mineralogy_pca_df = preproc.create_pca_df(mineralogy_pca, mineralogy_clr)

____

## Coordinates

In [383]:
coordinates

,Latitude,Longitude,past_mer,Y,X
1,"44°31'30.0""","138°37'30.0""",NaN,44.525000,138.625000
2,"54°12'10.0""","119°24'0.0""",NaN,54.202778,119.400000
3,"62°36'0.0""","155°36'0.0""",NaN,62.600000,155.600000
4,"61°35'0.0""","146°2'0.0""",NaN,61.583333,146.033333
5,"68°55'0.0""","164°24'0.0""",NaN,68.916667,164.400000
...,...,...,...,...,...
4655,"66°42'0.0""","164°23'0.0""",NaN,66.700000,164.383333
4656,"46°56'30.0""","137°5'3.0""",NaN,46.941667,137.084167
4657,"58°12'0.0""","138°12'0.0""",NaN,58.200000,138.200000
4658,"60°51'0.0""","147°31'0.0""",NaN,60.850000,147.516667


In [384]:
# Delete negative signs in "Longitude" column for dms2dec function to work properly
coordinates["Longitude"] = coordinates["Longitude"].str.replace("-", "")

In [385]:
# Include W in "Longitude" column
sum_ = 0

for index, row in coordinates.iterrows():
    if ("W" in str(row["past_mer"])) or ("w" in str(row["past_mer"])):
        coordinates.loc[index, "Longitude"] = row["Longitude"] + "W"
        
        sum_ += 1

In [386]:
# Check that all occurences of "W" or "w" are catched
assert sum_ == int(coordinates["past_mer"].value_counts())

### Convert from degrees to decimal format

In [387]:
coordinates["Y"] = coordinates.loc[:, "Latitude"].apply(cleaning.dms2dec)
coordinates["X"] = coordinates.loc[:, "Longitude"].apply(cleaning.dms2dec)

In [388]:
# Check
coordinates.loc[42, "X"]

149.38333333333333

In [389]:
coordinates

,Latitude,Longitude,past_mer,Y,X
1,"44°31'30.0""","138°37'30.0""",NaN,44.525000,138.625000
2,"54°12'10.0""","119°24'0.0""",NaN,54.202778,119.400000
3,"62°36'0.0""","155°36'0.0""",NaN,62.600000,155.600000
4,"61°35'0.0""","146°2'0.0""",NaN,61.583333,146.033333
5,"68°55'0.0""","164°24'0.0""",NaN,68.916667,164.400000
...,...,...,...,...,...
4655,"66°42'0.0""","164°23'0.0""",NaN,66.700000,164.383333
4656,"46°56'30.0""","137°5'3.0""",NaN,46.941667,137.084167
4657,"58°12'0.0""","138°12'0.0""",NaN,58.200000,138.200000
4658,"60°51'0.0""","147°31'0.0""",NaN,60.850000,147.516667


coordinates = coordinates.rename({"Y" : "Latitude"}, axis=1)
coordinates = coordinates.rename({"X" : "Longitude"}, axis=1)


In [390]:
coordinates

,Latitude,Longitude,past_mer,Y,X
1,"44°31'30.0""","138°37'30.0""",NaN,44.525000,138.625000
2,"54°12'10.0""","119°24'0.0""",NaN,54.202778,119.400000
3,"62°36'0.0""","155°36'0.0""",NaN,62.600000,155.600000
4,"61°35'0.0""","146°2'0.0""",NaN,61.583333,146.033333
5,"68°55'0.0""","164°24'0.0""",NaN,68.916667,164.400000
...,...,...,...,...,...
4655,"66°42'0.0""","164°23'0.0""",NaN,66.700000,164.383333
4656,"46°56'30.0""","137°5'3.0""",NaN,46.941667,137.084167
4657,"58°12'0.0""","138°12'0.0""",NaN,58.200000,138.200000
4658,"60°51'0.0""","147°31'0.0""",NaN,60.850000,147.516667


### plotting in Qgis does not work --> will look into it (something to do with qgis)

In [391]:
coordinates.to_excel("../_INTERPOLATION/coordinates_decimal.xlsx")

### Convert to UTM coordinates

**To do**
* Group samples into certain groups based on spatial distribution
* Recalculate utm coordinates based on fixed zone (fixed letter and number)

In [392]:
coordinates_utm = coordinates.apply(lambda row : utm.from_latlon(row["Y"], row["X"]), axis=1)
coordinates_utm = coordinates_utm.apply(pd.Series)
coordinates_utm.columns = ["Y_UTM", "X_UTM", "ZoneNumber", "ZoneLetter"]

In [393]:
coordinates_utm

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter
1,311272.566098,4.932930e+06,54,T
2,656538.925786,6.008743e+06,50,U
3,633468.918083,6.943713e+06,56,V
4,448679.859182,6.828145e+06,55,V
5,475912.972942,7.645188e+06,58,W
...,...,...,...,...
4655,472781.240413,7.398072e+06,58,W
4656,658620.016371,5.200790e+06,53,T
4657,335447.154681,6.454395e+06,54,V
4658,528075.823175,6.746190e+06,55,V


In [394]:
coordinates_utm["ZoneNumber"].value_counts()

53    971
49    802
50    689
54    655
55    279
56    232
58    174
60    151
1     139
59    133
48    132
57    130
52     90
2      40
51     39
47      3
Name: ZoneNumber, dtype: int64

In [395]:
coordinates_utm["ZoneLetter"].value_counts()

U    1875
W    1117
T     878
V     758
X      31
Name: ZoneLetter, dtype: int64

In [396]:
(coordinates_utm["ZoneNumber"].astype(str) + coordinates_utm["ZoneLetter"]).value_counts()

49U    759
50U    658
53T    643
55V    197
56V    196
54W    193
54V    178
53U    167
53W    148
1W     139
54T    129
60W    126
59W    125
54U    124
58W    111
55W     82
48U     77
58V     63
52W     58
57V     57
57W     56
48T     55
49T     40
2W      40
51U     38
56W     36
52U     32
54X     31
60V     25
50V     20
57U     17
53V     13
50T     11
59V      8
47U      3
49W      3
51V      1
dtype: int64

In [397]:
coordinates_utm.to_excel("../_INTERPOLATION/coordinates_UTM.xlsx")
cof= pd.read_excel("../_RESULTS/working_data.xlsx", index_col=0, usecols = lambda column : column not in ["Lat_deg", "Lat_min", "Lat_sec", "Long_deg", "Long_min", "Long_sec", "past_mer"] )

In [398]:
cof

,type_granite,time,massif,sampler,others,sampler+year
1,Granite leucogranitic,K2,NaN,NaN,NaN,"V.N.Musin,1970"
2,Granite leucocratic,Tr,NaN,V.I.Zhigalova,NaN,"E.A.Ivanov,1969"
3,Granite leucocratic coarse-grained,K2,Omsukchan massif,P.M.Bosek,NaN,"O.S.Gracheva,1948"
4,Granite leucocratic,K1,Buksandzhin massif,A.Kh.Brovtman,NaN,"A.F.Mikhaylov,1948"
5,Granite-porphyry micropegmatitic,K1,Attykveem massif,L.G.Semenova,0th.:S-0.16,"A.I.Sadovsky,1963"
...,...,...,...,...,...,...
4655,Diorite,K1,Egdegkych massif,NaN,Oth.:co2-0.12,"V.A.lgnat'ev,1964"
4656,Quartz diorite,K2,Verkhneplotnikovsky massif,NaN,NaN,"A.A.Syas'ko,1969"
4657,Diorite,J3,NaN,NaN,"Oth.:co2-0.02,so3-0.0l","N.N.Remizov,1967"
4658,Diorite,K2,NaN,NaN,NaN,"A.P.Osipov,1966"


In [399]:
coordinates_full = pd.concat([coordinates_utm, coordinates, mineralogy, cof], axis = 1)

In [400]:
coordinates_full.to_excel("../_INTERPOLATION/coordinates_full_data.xlsx")


##### grouping the data

In [401]:
areax = coordinates_full
areax["area"] = ""


In [402]:
areay1 = areax[areax["X"].between(0, 120)]
area1 = areay1[areay1["Y"].between(0, 58)]
area1["area"] = 1
area1.to_excel("../_INTERPOLATION/area1.xlsx")


C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [403]:
areay2 = areax[areax["X"].between(121, 142)]
area2 = areay2[areay2["Y"].between(42, 58.60)]
area2["area"] = 2
area2.to_excel("../_INTERPOLATION/area2.xlsx")


C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [404]:
areay3 = areax[areax["X"].between(126, 160.5)]
area3 = areay3[areay3["Y"].between(58.6, 75)]
area3["area"] = 3
area3.to_excel("../_INTERPOLATION/area3.xlsx")

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [405]:
areay4 = areax[areax["X"].between(160.5, 180)]
area4 = areay4[areay4["Y"].between(58, 75)]
area4["area"] = 4
area4.to_excel("../_INTERPOLATION/area4.xlsx")

In [406]:
areay5 = areax[areax["X"].between(-180, -165)]
area5 = areay5[areay5["Y"].between(63, 70)]
area5["area"] = 5
area5.to_excel("../_INTERPOLATION/area5.xlsx")

In [407]:
area12 = area1.append(area2)
area123 = area12.append(area3) 
area1234 = area123.append(area4) 
area = area1234.append(area5) 
area.to_excel("../_INTERPOLATION/area_subdivided.xlsx")

____

## Metadata

In [408]:
metadata = pd.read_excel("../_INTERPOLATION/coordinates_full_data.xlsx", index_col=0, usecols=[0, 25, 26, 27, 28, 29, 30])

In [409]:
metadata

,type_granite,time,massif,sampler,others,sampler+year
1,Granite leucogranitic,K2,NaN,NaN,NaN,"V.N.Musin,1970"
2,Granite leucocratic,Tr,NaN,V.I.Zhigalova,NaN,"E.A.Ivanov,1969"
3,Granite leucocratic coarse-grained,K2,Omsukchan massif,P.M.Bosek,NaN,"O.S.Gracheva,1948"
4,Granite leucocratic,K1,Buksandzhin massif,A.Kh.Brovtman,NaN,"A.F.Mikhaylov,1948"
5,Granite-porphyry micropegmatitic,K1,Attykveem massif,L.G.Semenova,0th.:S-0.16,"A.I.Sadovsky,1963"
...,...,...,...,...,...,...
4655,Diorite,K1,Egdegkych massif,NaN,Oth.:co2-0.12,"V.A.lgnat'ev,1964"
4656,Quartz diorite,K2,Verkhneplotnikovsky massif,NaN,NaN,"A.A.Syas'ko,1969"
4657,Diorite,J3,NaN,NaN,"Oth.:co2-0.02,so3-0.0l","N.N.Remizov,1967"
4658,Diorite,K2,NaN,NaN,NaN,"A.P.Osipov,1966"


In [410]:
metadata["type_granite"].value_counts()

Granite                                               656
Granodiorite                                          523
Granite-porphyry                                      282
Bt granite                                            268
Granite leucocratic                                   220
                                                     ... 
Granite porphyraceous leucogranitic coarse-grained      1
Granite leucogranitic pegmatoid                         1
Granite monzonitic                                      1
Hb granodiorite-porphyry                                1
Granite granophyric subalkaline                         1
Name: type_granite, Length: 401, dtype: int64

In [411]:
metadata["massif"].value_counts()

Ulakhan-Sis massif         58
Khoboyotuu-Echiy massif    56
Vladimirsky massif         35
Zimov'e massif             33
Bom-Gorkhon massif         31
                           ..
Dogdin massif               1
Tas massif                  1
Choatakchin massif          1
Osinovsky massif            1
Buordakh massif             1
Name: massif, Length: 942, dtype: int64

In [412]:
metadata["time"].value_counts()

K2       1271
K1       1151
Tr        516
J3        411
Tr-J      338
Pg        193
K2-Pg     171
J2        154
J1-J2     109
Mz         88
J1         69
K          62
J          46
J-K        37
J2-J3      35
Name: time, dtype: int64

In [413]:
metadata["sampler"].value_counts()

L.S.Voronova      120
D.M.Shuster        87
N.A.Lebedeva       86
V.I.Zhigalova      70
N.P.Mel'nikova     51
                 ... 
R.N.Il'nitsky       1
Z.IKondrashina      1
ANIl'inykh.         1
N.SSergutina        1
E.Gerasimova        1
Name: sampler, Length: 736, dtype: int64

In [414]:
metadata["sampler+year"].value_counts()

G.A.Valuy,1975          76
V.A.Popeko,1968         61
V.A.Faradzhev,1971      43
V.S.Ivanov,1968         37
R.O.Galabala,1976       37
                        ..
A.V.Ivanov,1956          1
F.A.Golovachev,1935      1
K.F.Khatskevich,1966     1
I.P.Boyko,1966           1
V.I.Goldenberg,1958      1
Name: sampler+year, Length: 1392, dtype: int64

## Saving of data

In [415]:
# Save data as pickle files to use them in later notebooks
preproc.save_obj(mineralogy, "mineralogy") # mineralogy
preproc.save_obj(mineralogy_clr, "mineralogy_clr") # mineralogy clr
preproc.save_obj(mineralogy_pca, "mineralogy_pca") # mineralogy pca info
preproc.save_obj(mineralogy_pca_df, "mineralogy_pca_df") # mineralogy pca scores

preproc.save_obj(coordinates, "coordinates") # coordinates latlon
preproc.save_obj(coordinates_utm, "coordinates_utm") # coordinates utm
preproc.save_obj(metadata, "metadata") # metadata

____